In [7]:
# INITSYNCH --- function that fetch video and mocap and the respective points of synchronization
#                       Input:
#                           - action: name of the action from synch_index.csv file
#                           - mode: indicates if the action belongs to the
#                             training set ('tr') or the test set ('te'),
#                             or if it is a test scene ('scene')
#                           - PoV: indicates which point of view we want to
#                             visualize along with the MoCap data, 
#                                   0 (from the right)
#                                   1 (actor PoV)
#                                   2 (from the front)
#                           - mocap_folder: path of the folder containing
#                             the MoCap data
#                           - video_folder: path of the folder containing
#                             the video
#
#                       Output:
#                           - vid, a VideoReader object
#                           - mocap, the stream of motion capture data
#                           - startMocap/startVid starting frames for the
#                             synchronization
#                           - pov, one of the three point of views
#                           
# Example of use:
# action_list=['carrot','cutting','dish','eating','eggs','lemon','mezzaluna','mixing',
#'openbottle','pan','pestling','pouringmult','pouringsingle','reaching','rolling',
# 'salad','salting','spreading','table','transporting',
# 'scene1','scene2','scene3','scene4','scene5']

#mocap_folder = '../CookingDataset-master/data/mocap/';
#video_folder = '../CookingDataset-master/data/video/';   
#action="carrot"  # select from action_list
#mode="tr"
#pov=0
#vid, mocap, startMocap, startVid, pov=intSynch(action, mode, pov, mocap_folder, video_folder)

# Use the output as input for the function synchronizedView to visualize
# video and skeleton in parallel.
# synchronizedView(vid, mocap, startMocap, startVide, pov);

import os
from os import path
import time
import sys
import csv
import pandas as pd
from scipy.io import loadmat 
import pylab
import cv2 

def intSynch(action, mode, pov, mocap_folder, video_folder):
    synch_indexfile="synch_index.csv"
    df_synchindex=pd.read_csv(synch_indexfile)

    df_row=df_synchindex.loc[df_synchindex['action'] == action].values.tolist()#[['carrot', 1500, 90, 601, 76]]
    
    if mode=='tr':
        startMocap = df_row[0][1]
        startVid =   df_row[0][2]
        mocap_file=path.join(mocap_folder,"training",action+"_"+mode+".mat")
        video_file=path.join(video_folder,"training",action.upper()+"_"+mode.upper()+"_"+str(pov)+".avi")

    elif mode == 'te':
        startMocap = df_row[0][3]
        startVid =   df_row[0][4]
        mocap_file=path.join(mocap_folder,"test",action+"_test.mat")
        video_file=path.join(video_folder,"test",action.upper()+"_"+mode.upper()+"_"+str(pov)+".avi")
    elif mode == 'scene':
        startMocap = df_row[0][1]
        startVid =   df_row[0][2]
        mocap_file=path.join(mocap_folder,"scenes",action.capitalize()+".mat")
        video_file=path.join(video_folder,"scenes",action.upper()+"_"+str(pov)+".avi")
    

    
    print("video_file:",video_file)
    vid = cv2.VideoCapture(video_file)
    
    #motion capture
    column_names=['IND_X','IND_Y','IND_Z','PALM_X','PALM_Y','PALM_Z', 'LIT_X', 'LIT_Y','LIT_Z','WRIST_X','WRIST_Y','WRIST_Z', 'ELBOW_X','ELBOW_Y','ELBOW_Z', 'SHOULDER_X','SHOULDER_Y','SHOULDER_Z']
    df_mocap = pd.DataFrame(columns = column_names)

    single_action = loadmat(mocap_file)  # load mat-file in python

    arr_index=single_action['index'] # index array
    start_index=arr_index[0][0] # start index

    
    marker_list = ['IND','PALM', 'LIT', 'WRIST', 'ELBOW', 'SHOULDER']
    for marker in marker_list:
        arr = single_action[marker]

        x=marker+'_X'
        y=marker+'_Y'
        z=marker+'_Z'
        df_mocap[x]=arr[:,0]
        df_mocap[y]=arr[:,1]
        df_mocap[z]=arr[:,2]
    
    mocap=df_mocap
    return vid, mocap, startMocap, startVid, pov

In [8]:
# Function Calling
mocap_folder = '../CookingDataset-master/data/mocap/';
video_folder = '../CookingDataset-master/data/video/';   
# action="carrot"
# mode="tr"
  
action="scene1" # select actions from index file (synch_index.csv)
mode="scene"
pov=0

vid, mocap, startMocap, startVid, pov=intSynch(action, mode, pov, mocap_folder, video_folder)
print("vid:",vid)
# print(mocap)
print("startMocap:",startMocap)
print("startVid:",startVid)
# print("pov:",pov)

video_file: ../CookingDataset-master/data/video/scenes/SCENE1_0.avi
vid: <VideoCapture 0x7f908d63fdf0>
startMocap: 1165
startVid: 120
